**Описание данных**: таблица game_sales с количеством проданных игр с 1977 года (400 видеоигр) и reviews c оценки критиков и пользователей видеоигр.

**План решения**:
1. Из таблицы game_sold выведем десять продаваемых видеоигр
2. Посчитаем количество игр из таблицы game_sold без оценок критикой и пользователей (таблица reviews)
3. Year, которые понравились критикам видеоигр
4. Скорректируем на количество выпускаемых игр
5. Выведем year где меньше 4 игр
6. Year, которые понравились игрокам видеоигр
7. Year, которые понравились и игрокам, и критикам (пересечение из топ 10 тех и других)
8. Количество продаж по годам из пункта 7

**Инструменты**: LEFT JOIN, INNER JOIN, HAVING, EXCEPT, INTERSECT

In [ ]:
%%sql
postgresql:///games

SELECT *
FROM game_sales
ORDER BY games_sold DESC
LIMIT 10;

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [ ]:
%%sql

SELECT COUNT(*)
FROM game_sales
LEFT JOIN reviews
USING (game)
WHERE critic_score IS NULL AND user_score IS NULL

 * postgresql:///games
1 rows affected.


count
31


In [ ]:
%%sql

SELECT year, COUNT(game), ROUND(AVG(critic_score),2) AS avg_critic_score
FROM game_sales
INNER JOIN reviews
USING (game)
GROUP BY year
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,count,avg_critic_score
1990,1,9.80
1992,3,9.67
1998,10,9.32
2020,2,9.20
1993,2,9.10
1995,3,9.07
2004,11,9.03
1982,1,9.00
2002,9,8.99
1999,11,8.93


In [ ]:
%%sql

SELECT year, COUNT(game) AS num_games, ROUND(AVG(critic_score),2) AS avg_critic_score
FROM game_sales
INNER JOIN reviews
USING (game)
GROUP BY year
HAVING COUNT(game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,num_games,avg_critic_score
1998,10,9.32
2004,11,9.03
2002,9,8.99
1999,11,8.93
2001,13,8.82
2011,26,8.76
2016,13,8.67
2013,18,8.66
2008,20,8.63
2017,13,8.62


In [ ]:
%%sql

SELECT year, avg_critic_score
FROM top_critic_years
EXCEPT
SELECT year, avg_critic_score
FROM top_critic_years_more_than_four_games
ORDER BY avg_critic_score DESC;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


In [ ]:
%%sql

SELECT year, COUNT(game) AS num_games, ROUND(AVG(user_score),2) AS avg_user_score
FROM game_sales
INNER JOIN reviews
USING (game)
GROUP BY year
HAVING COUNT(game) > 4
ORDER BY avg_user_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,num_games,avg_user_score
1997,8,9.50
1998,10,9.40
2010,23,9.24
2009,20,9.18
2008,20,9.03
1996,5,9.00
2005,13,8.95
2006,16,8.95
2000,8,8.80
2002,9,8.80


In [ ]:
%%sql

SELECT year
FROM (SELECT year, COUNT(game) AS num_games, ROUND(AVG(user_score),2) AS avg_user_score
      FROM game_sales
      INNER JOIN reviews
      USING (game)
      GROUP BY year
      HAVING COUNT(game) > 4
      ORDER BY avg_user_score DESC
      LIMIT 10) as t1
INTERSECT
SELECT year
FROM (SELECT year, COUNT(game), ROUND(AVG(critic_score),2) AS avg_critic_score
      FROM game_sales
      INNER JOIN reviews
      USING (game)
      GROUP BY year
      HAVING COUNT(game) > 4
      ORDER BY avg_critic_score DESC
      LIMIT 10) as t2
ORDER BY year ASC;

 * postgresql:///games
3 rows affected.


year
1998
2002
2008


In [ ]:
%%sql

SELECT year, SUM(games_sold) AS total_games_sold
FROM game_sales
WHERE year IN (SELECT year
               FROM top_user_years_more_than_four_games
               INTERSECT
               SELECT year
               FROM top_critic_years_more_than_four_games)
GROUP BY year
ORDER BY total_games_sold DESC;

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
